# Automatyzacja Excela

# Pierwsze kroki z xlwings

## Excel jako przeglądarka danych

In [ ]:
# Najpierw importujemy pakiety, których będziemy używać w tym rozdziale
import datetime as dt
import xlwings as xw
import pandas as pd
import numpy as np

In [ ]:
# Tworzymy DataFrame na podstawie liczb pseudolosowych, z taką
# liczbą wierszy, że widoczne będą tylko początkowe i końcowe
df = pd.DataFrame(data=np.random.randn(100, 5),
                  columns=[f"Próba {i}" for i in range(1, 6)])
df

In [ ]:
# Wyświetlenie DataFrame w Excelu
xw.view(df)

## Model obiektowy Excela

In [ ]:
# Utworzenie nowego pustego skoroszytu i wypisanie jego nazwy. Tego skoroszytu 
# będziemy używać do uruchamiania większości przykładów kodu z tego rozdziału.
book = xw.Book()
book.name

In [ ]:
# Uzyskanie dostępu do kolekcji arkuszy
book.sheets

In [ ]:
# Uzyskanie obiektu arkusza poprzez indeks lub nazwę. Jeśli
# Twój arkusz ma inną nazwę, wpisz ją zamiast "Arkusz1".
sheet1 = book.sheets[0]
sheet1 = book.sheets["Arkusz1"]

In [ ]:
sheet1.range("A1")

In [ ]:
# Najczęstsze zadania: zapisywanie wartości ...
sheet1.range("A1").value = [[1, 2],
                            [3, 4]]
sheet1.range("A4").value = "Witaj!"

In [ ]:
# ... i odczytywanie wartości
sheet1.range("A1:B2").value

In [ ]:
sheet1.range("A4").value

In [ ]:
# Indeksowanie
sheet1.range("A1:B2")[0, 0]

In [ ]:
# Wycinanie
sheet1.range("A1:B2")[:, 1]

In [ ]:
# Pojedyncza komórka: notacja A1
sheet1["A1"]

In [ ]:
# Wiele komórek: notacja A1
sheet1["A1:B2"]

In [ ]:
# Pojedyncza komórka: indeksowanie
sheet1[0, 0]

In [ ]:
# Wiele komórek: wycinanie
sheet1[:2, :2]

In [ ]:
# D10 poprzez indeksowanie arkusza
sheet1[9, 3]

In [ ]:
# D10 poprzez obiekt range
sheet1.range((10, 4))

In [ ]:
# D10:F11 poprzez wycinanie arkusza
sheet1[9:11, 3:6]

In [ ]:
# D10:F11 poprzez obiekt range
sheet1.range((10, 4), (11, 6))

In [ ]:
sheet1["A1"].sheet.book.app

In [ ]:
# Pobranie jednego obiektu app z otwartego skoroszytu
# i utworzenie dodatkowej niewidocznej instancji app
visible_app = sheet1.book.app
invisible_app = xw.App(visible=False)

In [ ]:
# Wypisanie nazw skoroszytów, które są otwarte w każdej instancji,
# poprzez wykorzystanie listy składanej
[book.name for book in visible_app.books]

In [ ]:
[book.name for book in invisible_app.books]

In [ ]:
# Klucz aplikacji, reprezentujący identyfikator procesu (PID)
xw.apps.keys()

In [ ]:
# Można uzyskać do niego dostęp również poprzez atrybut pid
xw.apps.active.pid

In [ ]:
# Praca ze skoroszytem w niewidocznej instancji Excela
invisible_book = invisible_app.books[0]
invisible_book.sheets[0]["A1"].value = "Utworzone przez niewidoczną aplikację."

In [ ]:
# Zapisanie skoroszytu Excela w katalogu xl
invisible_book.save("xl/invisible.xlsx")

In [ ]:
# Zamknięcie niewidocznej instancji Excela
invisible_app.quit()

## Uruchamianie kodu VBA

In [ ]:
vba_book = xw.Book("xl/vba.xlsm")

In [ ]:
# Utworzenie instancji obiektu macro z funkcją VBA
mysum = vba_book.macro("Module1.MySum")
# Wywołanie funkcji VBA
mysum(5, 4)

In [ ]:
# Tak samo działa to z procedurą Sub VBA
show_msgbox = vba_book.macro("Module1.ShowMsgBox")
show_msgbox("Witaj, xlwings!")

In [ ]:
# Zamknięcie skoroszytu (upewnij się, że najpierw zamknąłeś okno MessageBox)
vba_book.close()

# Konwertery, opcje i kolekcje

## Praca z obiektami DataFrame

In [ ]:
data=[["Mark", 55, "Włochy", 4.5, "Europa"],
      ["John", 33, "USA", 6.7, "Ameryka"]]
df = pd.DataFrame(data=data,
                  columns=["imię", "wiek", "kraj",
                           "ocena", "kontynent"],
                  index=[1001, 1000])
df.index.name = "numer"
df

In [ ]:
sheet1["A6"].value = df

In [ ]:
sheet1["B10"].options(header=False, index=False).value = df

In [ ]:
df2 = sheet1["A6"].expand().options(pd.DataFrame).value
df2

In [ ]:
# Jeśli chcesz, aby indeks zawierał liczby całkowite,
# możesz zmienić jego typ danych
df2.index = df2.index.astype(int)
df2

In [ ]:
# Ustawienie index=False spowoduje, że wszystkie wartości z Excela zostaną
# umieszczone w części danych DataFrame i zostanie użyty domyślny indeks
sheet1["A6"].expand().options(pd.DataFrame, index=False).value

## Konwertery i opcje

In [ ]:
# Zakres poziomy (jednowymiarowy)
sheet1["A1:B1"].value

In [ ]:
# Zakres pionowy (jednowymiarowy)
sheet1["A1:A2"].value

In [ ]:
# Zakres poziomy (dwuwymiarowy)
sheet1["A1:B1"].options(ndim=2).value

In [ ]:
# Zakres pionowy (dwuwymiarowy)
sheet1["A1:A2"].options(ndim=2).value

In [ ]:
# Taki sam efekt da użycie konwertera do tablic NumPy:
# pionowy zakres prowadzi do tablicy jednowymiarowej
sheet1["A1:A2"].options(np.array).value

In [ ]:
# Zachowanie orientacji kolumn
sheet1["A1:A2"].options(np.array, ndim=2).value

In [ ]:
# Jeśli musisz wypisać listę w pionie,
# przydatna będzie opcja "transpose".
sheet1["D1"].options(transpose=True).value = [100, 200]

In [ ]:
# Wypisz jakieś przykładowe dane
sheet1["A13"].value = [dt.datetime(2020, 1, 1), None, 1.0]

In [ ]:
# Wczytaj je z powrotem, używając domyślnych opcji
sheet1["A13:C13"].value

In [ ]:
# Wczytaj je ponownie, używając opcji innych niż domyślne
sheet1["A13:C13"].options(empty="nd.",
                          dates=dt.date,
                          numbers=int).value

## Wykresy, obrazy i zdefiniowane nazwy

In [ ]:
sheet1["A15"].value = [[None, "Północ", "Południe"],
                       ["Ubiegły rok", 2, 5],
                       ["Bieżący rok", 3, 6]]

In [ ]:
chart = sheet1.charts.add(top=sheet1["A19"].top,
                          left=sheet1["A19"].left)
chart.chart_type = "column_clustered"
chart.set_source_data(sheet1["A15"].expand())

In [ ]:
# Wczytanie danych wykresu jako DataFrame
df = sheet1["A15"].expand().options(pd.DataFrame).value
df

In [ ]:
# Włączenie Matplotlib przy użyciu magicznego polecenia
# i przełączenie na styl "seaborn"
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
# Metoda plot biblioteki pandas zwraca obiekt "axis", z którego
# można uzyskać obiekt figure. "T" wykonuje transpozycję DataFrame,
# aby ustawić wykres w żądanej orientacji
ax = df.T.plot.bar()
fig = ax.get_figure()

In [ ]:
# Wysłanie wykresu do Excela.
plot = sheet1.pictures.add(fig, name="SalesPlot",
                           top=sheet1["H19"].top,
                           left=sheet1["H19"].left)
# Przeskalowanie wykresu do 70%
plot.width, plot.height = plot.width * 0.7, plot.height * 0.7

In [ ]:
ax = (df + 1).T.plot.bar()
plot = plot.update(ax.get_figure())

In [ ]:
# Zakresem domyślnym jest zakres skoroszytu
sheet1["A1:B2"].name = "macierz1"

In [ ]:
# W przypadku zakresu arkusza po nazwie arkusza
# należy wstawić wykrzyknik
sheet1["B10:E11"].name = "Arkusz1!macierz2"

In [ ]:
# Teraz możesz uzyskać dostęp do zakresu poprzez nazwę
sheet1["macierz1"]

In [ ]:
# Jeśli dostęp do kolekcji nazw odbywa się poprzez obiekt "sheet1",
# zawiera ona tylko nazwy w obrębie tego arkusza
sheet1.names

In [ ]:
# Jeśli dostęp do kolekcji nazw odbywa się poprzez obiekt "book",
# zawiera ona wszystkie nazwy, zarówno w obrębie skoroszytu, jak i arkusza
book.names

In [ ]:
# Nazwy mają różne metody i atrybuty.
# Dzięki temu możesz np. uzyskać odpowiedni obiekt zakresu.
book.names["macierz1"].refers_to_range

In [ ]:
# Jeśli chcesz przypisać nazwę do stałej
# lub formuły, użyj metody "add"
book.names.add("EURUSD", "=1,1151")

# Zagadnienia zaawansowane

## Wydajność

In [ ]:
# Dodanie nowego arkusza i wpisanie do niego
# 150 wartości, by mieć z czym pracować
sheet2 = book.sheets.add()
sheet2["A1"].value = np.arange(150).reshape(30, 5)

In [ ]:
%%time
# W ten sposób powstaje 150 wywołań międzyaplikacyjnych
for cell in sheet2["A1:E30"]:
    cell.value += 1

In [ ]:
%%time
# W ten sposób powstają tylko dwa wywołania międzyaplikacyjne
values = sheet2["A1:E30"].options(np.array).value
sheet2["A1"].value = values + 1

In [ ]:
# W przypadku surowych wartości musisz podać pełny
# zakres docelowy, sheet["A35"] już nie zadziała
sheet1["A35:B36"].options("raw").value = [[1, 2], [3, 4]]